In [1]:
import numpy as np 
import pandas as pd
import pickle

In [2]:
%cd C:\Users\welcome\Downloads\Face Recognition

C:\Users\welcome\Downloads\Face Recognition


In [3]:
data=pickle.load(open('data_face_features.pickle',mode='rb'))

In [4]:
data

{'data': [array([[-0.04815767,  0.19590802, -0.08565018, -0.00674671, -0.0865007 ,
           0.22394715, -0.0403205 , -0.02719629,  0.02758096,  0.04676095,
           0.14919397,  0.02814778,  0.06758796, -0.12510157, -0.05817988,
          -0.16143787, -0.09448455,  0.09368823, -0.03454525,  0.085752  ,
           0.01062765, -0.05017038, -0.03253431,  0.08869503, -0.02247124,
          -0.06664498,  0.00663711, -0.0774919 ,  0.03987211,  0.05796074,
           0.08230448, -0.07575367,  0.0397023 ,  0.03501398,  0.007204  ,
           0.08006705, -0.01172677, -0.02581719, -0.079477  ,  0.01713717,
           0.04615965, -0.18087684, -0.06816249,  0.16546822, -0.17272452,
          -0.16195525,  0.11863638,  0.0567521 , -0.13177146,  0.0661111 ,
           0.01912695,  0.07856851, -0.07569704, -0.00124933, -0.00688392,
           0.11573324, -0.12773728,  0.02371483, -0.01212493, -0.12556006,
          -0.17726952, -0.02327819,  0.23309201, -0.14731245,  0.02722122,
          -0.0614

In [5]:
X=np.array(data['data'])
Y=np.array(data['label'])

In [6]:
X.shape,Y.shape

((695, 1, 128), (695,))

In [7]:
X=X.reshape(-1,128)

In [8]:
X.shape

(695, 128)

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
X_train,X_test,y_train,y_test=train_test_split(X,Y,train_size=0.8,random_state=0)

In [11]:
X_train.shape

(556, 128)

In [12]:
X_test.shape

(139, 128)

## Train the machine learning model 

In [13]:
from sklearn.linear_model import LogisticRegression 
from sklearn.svm import SVC 
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier 
from sklearn.metrics import classification_report,accuracy_score,f1_score

## Logistic Regression Model


In [14]:
model_logistic=LogisticRegression()

In [15]:
model_logistic.fit(X_train,y_train)


LogisticRegression()

In [16]:
y_pred_train=model_logistic.predict(X_train) 
y_pred_test=model_logistic.predict(X_test) 
 
#accuracy score 
acc_train=accuracy_score(y_train,y_pred_train)
acc_test=accuracy_score(y_test,y_pred_test)


In [17]:
#f1 score 
f1_score_train=f1_score(y_train,y_pred_train,average='macro') 
f1_score_test=f1_score(y_test,y_pred_test,average='macro')

In [18]:
acc_train

0.8093525179856115

In [19]:
acc_test

0.7482014388489209

In [20]:
f1_score_train

0.8066878004582234

In [21]:
f1_score_test

0.7386529515842767

In [22]:
def get_report(model,X_train,y_train,X_test,y_test):
    y_pred_train=model.predict(X_train) 
    y_pred_test=model.predict(X_test) 
 
    #accuracy score 
    acc_train=accuracy_score(y_train,y_pred_train)
    acc_test=accuracy_score(y_test,y_pred_test) 
    
    #f1 score 
    f1_score_train=f1_score(y_train,y_pred_train,average='macro') 
    f1_score_test=f1_score(y_test,y_pred_test,average='macro') 
    
    print('Accuracy score of training data=%0.2f'%acc_train)
    print('Accuracy score of testing data=%0.2f'%acc_test) 
    print('F1 score of training_data=%0.2f'%f1_score_train) 
    print('F1 score of training_data=%0.2f'%f1_score_test)

In [23]:
get_report(model_logistic,X_train,y_train,X_test,y_test)

Accuracy score of training data=0.81
Accuracy score of testing data=0.75
F1 score of training_data=0.81
F1 score of training_data=0.74


In [24]:
model_svc=SVC(probability=True)
model_svc.fit(X_train,y_train)

SVC(probability=True)

In [25]:
get_report(model_svc,X_train,y_train,X_test,y_test)

Accuracy score of training data=0.88
Accuracy score of testing data=0.77
F1 score of training_data=0.88
F1 score of training_data=0.77


In [26]:
model_rf=RandomForestClassifier(n_estimators=5) 
model_rf.fit(X_train,y_train)

RandomForestClassifier(n_estimators=5)

In [27]:
get_report(model_rf,X_train,y_train,X_test,y_test)

Accuracy score of training data=0.97
Accuracy score of testing data=0.58
F1 score of training_data=0.97
F1 score of training_data=0.58


## Voting Classifier

In [28]:
model_voting=VotingClassifier(estimators=[
    ('logistic',LogisticRegression()),
    ('svm',SVC(probability=True)),
    ('rf',RandomForestClassifier())
],voting='soft',weights=[2,3,1])

In [29]:
model_voting.fit(X_train,y_train)

VotingClassifier(estimators=[('logistic', LogisticRegression()),
                             ('svm', SVC(probability=True)),
                             ('rf', RandomForestClassifier())],
                 voting='soft', weights=[2, 3, 1])

In [30]:
get_report(model_voting,X_train,y_train,X_test,y_test)

Accuracy score of training data=0.92
Accuracy score of testing data=0.78
F1 score of training_data=0.92
F1 score of training_data=0.77


## Paramter Tuning 

In [31]:
from sklearn.model_selection import GridSearchCV

In [32]:
model_grid=GridSearchCV(model_voting,param_grid={
    'svm__C':[3,5,7,9],
    'svm__gamma':[0.1,0.3,0.5],
    'rf__n_estimators':[5,10,20],
    'rf__max_depth':[3,5,7],
    'voting':['soft','hard'],
},scoring='accuracy',cv=3,n_jobs=1,verbose=2)

In [33]:
model_grid.fit(X_train,y_train)

Fitting 3 folds for each of 216 candidates, totalling 648 fits
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=soft; total time=   0.0s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=soft; total time=   0.0s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=soft; total time=   0.0s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=hard; total time=   0.1s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=hard; total time=   0.0s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=hard; total time=   0.0s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.3, voting=soft; total time=   0.0s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.3, voting=soft; total time=   0.0s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.3, voting=soft; total time=   0.0s
[

[CV] END rf__max_depth=3, rf__n_estimators=10, svm__C=3, svm__gamma=0.3, voting=soft; total time=   0.1s
[CV] END rf__max_depth=3, rf__n_estimators=10, svm__C=3, svm__gamma=0.3, voting=hard; total time=   0.0s
[CV] END rf__max_depth=3, rf__n_estimators=10, svm__C=3, svm__gamma=0.3, voting=hard; total time=   0.0s
[CV] END rf__max_depth=3, rf__n_estimators=10, svm__C=3, svm__gamma=0.3, voting=hard; total time=   0.1s
[CV] END rf__max_depth=3, rf__n_estimators=10, svm__C=3, svm__gamma=0.5, voting=soft; total time=   0.1s
[CV] END rf__max_depth=3, rf__n_estimators=10, svm__C=3, svm__gamma=0.5, voting=soft; total time=   0.1s
[CV] END rf__max_depth=3, rf__n_estimators=10, svm__C=3, svm__gamma=0.5, voting=soft; total time=   0.1s
[CV] END rf__max_depth=3, rf__n_estimators=10, svm__C=3, svm__gamma=0.5, voting=hard; total time=   0.1s
[CV] END rf__max_depth=3, rf__n_estimators=10, svm__C=3, svm__gamma=0.5, voting=hard; total time=   0.1s
[CV] END rf__max_depth=3, rf__n_estimators=10, svm__C=3

[CV] END rf__max_depth=3, rf__n_estimators=20, svm__C=3, svm__gamma=0.5, voting=hard; total time=   0.1s
[CV] END rf__max_depth=3, rf__n_estimators=20, svm__C=3, svm__gamma=0.5, voting=hard; total time=   0.0s
[CV] END rf__max_depth=3, rf__n_estimators=20, svm__C=3, svm__gamma=0.5, voting=hard; total time=   0.1s
[CV] END rf__max_depth=3, rf__n_estimators=20, svm__C=5, svm__gamma=0.1, voting=soft; total time=   0.0s
[CV] END rf__max_depth=3, rf__n_estimators=20, svm__C=5, svm__gamma=0.1, voting=soft; total time=   0.0s
[CV] END rf__max_depth=3, rf__n_estimators=20, svm__C=5, svm__gamma=0.1, voting=soft; total time=   0.0s
[CV] END rf__max_depth=3, rf__n_estimators=20, svm__C=5, svm__gamma=0.1, voting=hard; total time=   0.1s
[CV] END rf__max_depth=3, rf__n_estimators=20, svm__C=5, svm__gamma=0.1, voting=hard; total time=   0.1s
[CV] END rf__max_depth=3, rf__n_estimators=20, svm__C=5, svm__gamma=0.1, voting=hard; total time=   0.1s
[CV] END rf__max_depth=3, rf__n_estimators=20, svm__C=5

[CV] END rf__max_depth=5, rf__n_estimators=5, svm__C=5, svm__gamma=0.1, voting=hard; total time=   0.0s
[CV] END rf__max_depth=5, rf__n_estimators=5, svm__C=5, svm__gamma=0.1, voting=hard; total time=   0.0s
[CV] END rf__max_depth=5, rf__n_estimators=5, svm__C=5, svm__gamma=0.3, voting=soft; total time=   0.0s
[CV] END rf__max_depth=5, rf__n_estimators=5, svm__C=5, svm__gamma=0.3, voting=soft; total time=   0.0s
[CV] END rf__max_depth=5, rf__n_estimators=5, svm__C=5, svm__gamma=0.3, voting=soft; total time=   0.0s
[CV] END rf__max_depth=5, rf__n_estimators=5, svm__C=5, svm__gamma=0.3, voting=hard; total time=   0.0s
[CV] END rf__max_depth=5, rf__n_estimators=5, svm__C=5, svm__gamma=0.3, voting=hard; total time=   0.0s
[CV] END rf__max_depth=5, rf__n_estimators=5, svm__C=5, svm__gamma=0.3, voting=hard; total time=   0.0s
[CV] END rf__max_depth=5, rf__n_estimators=5, svm__C=5, svm__gamma=0.5, voting=soft; total time=   0.0s
[CV] END rf__max_depth=5, rf__n_estimators=5, svm__C=5, svm__gam

[CV] END rf__max_depth=5, rf__n_estimators=10, svm__C=5, svm__gamma=0.3, voting=hard; total time=   0.1s
[CV] END rf__max_depth=5, rf__n_estimators=10, svm__C=5, svm__gamma=0.5, voting=soft; total time=   0.1s
[CV] END rf__max_depth=5, rf__n_estimators=10, svm__C=5, svm__gamma=0.5, voting=soft; total time=   0.1s
[CV] END rf__max_depth=5, rf__n_estimators=10, svm__C=5, svm__gamma=0.5, voting=soft; total time=   0.1s
[CV] END rf__max_depth=5, rf__n_estimators=10, svm__C=5, svm__gamma=0.5, voting=hard; total time=   0.0s
[CV] END rf__max_depth=5, rf__n_estimators=10, svm__C=5, svm__gamma=0.5, voting=hard; total time=   0.0s
[CV] END rf__max_depth=5, rf__n_estimators=10, svm__C=5, svm__gamma=0.5, voting=hard; total time=   0.0s
[CV] END rf__max_depth=5, rf__n_estimators=10, svm__C=7, svm__gamma=0.1, voting=soft; total time=   0.0s
[CV] END rf__max_depth=5, rf__n_estimators=10, svm__C=7, svm__gamma=0.1, voting=soft; total time=   0.1s
[CV] END rf__max_depth=5, rf__n_estimators=10, svm__C=7

[CV] END rf__max_depth=5, rf__n_estimators=20, svm__C=7, svm__gamma=0.1, voting=soft; total time=   0.1s
[CV] END rf__max_depth=5, rf__n_estimators=20, svm__C=7, svm__gamma=0.1, voting=soft; total time=   0.1s
[CV] END rf__max_depth=5, rf__n_estimators=20, svm__C=7, svm__gamma=0.1, voting=soft; total time=   0.1s
[CV] END rf__max_depth=5, rf__n_estimators=20, svm__C=7, svm__gamma=0.1, voting=hard; total time=   0.0s
[CV] END rf__max_depth=5, rf__n_estimators=20, svm__C=7, svm__gamma=0.1, voting=hard; total time=   0.0s
[CV] END rf__max_depth=5, rf__n_estimators=20, svm__C=7, svm__gamma=0.1, voting=hard; total time=   0.1s
[CV] END rf__max_depth=5, rf__n_estimators=20, svm__C=7, svm__gamma=0.3, voting=soft; total time=   0.1s
[CV] END rf__max_depth=5, rf__n_estimators=20, svm__C=7, svm__gamma=0.3, voting=soft; total time=   0.1s
[CV] END rf__max_depth=5, rf__n_estimators=20, svm__C=7, svm__gamma=0.3, voting=soft; total time=   0.0s
[CV] END rf__max_depth=5, rf__n_estimators=20, svm__C=7

[CV] END rf__max_depth=7, rf__n_estimators=5, svm__C=7, svm__gamma=0.3, voting=soft; total time=   0.1s
[CV] END rf__max_depth=7, rf__n_estimators=5, svm__C=7, svm__gamma=0.3, voting=soft; total time=   0.0s
[CV] END rf__max_depth=7, rf__n_estimators=5, svm__C=7, svm__gamma=0.3, voting=hard; total time=   0.0s
[CV] END rf__max_depth=7, rf__n_estimators=5, svm__C=7, svm__gamma=0.3, voting=hard; total time=   0.1s
[CV] END rf__max_depth=7, rf__n_estimators=5, svm__C=7, svm__gamma=0.3, voting=hard; total time=   0.0s
[CV] END rf__max_depth=7, rf__n_estimators=5, svm__C=7, svm__gamma=0.5, voting=soft; total time=   0.0s
[CV] END rf__max_depth=7, rf__n_estimators=5, svm__C=7, svm__gamma=0.5, voting=soft; total time=   0.0s
[CV] END rf__max_depth=7, rf__n_estimators=5, svm__C=7, svm__gamma=0.5, voting=soft; total time=   0.0s
[CV] END rf__max_depth=7, rf__n_estimators=5, svm__C=7, svm__gamma=0.5, voting=hard; total time=   0.1s
[CV] END rf__max_depth=7, rf__n_estimators=5, svm__C=7, svm__gam

[CV] END rf__max_depth=7, rf__n_estimators=10, svm__C=7, svm__gamma=0.5, voting=soft; total time=   0.2s
[CV] END rf__max_depth=7, rf__n_estimators=10, svm__C=7, svm__gamma=0.5, voting=hard; total time=   0.2s
[CV] END rf__max_depth=7, rf__n_estimators=10, svm__C=7, svm__gamma=0.5, voting=hard; total time=   0.2s
[CV] END rf__max_depth=7, rf__n_estimators=10, svm__C=7, svm__gamma=0.5, voting=hard; total time=   0.3s
[CV] END rf__max_depth=7, rf__n_estimators=10, svm__C=9, svm__gamma=0.1, voting=soft; total time=   0.4s
[CV] END rf__max_depth=7, rf__n_estimators=10, svm__C=9, svm__gamma=0.1, voting=soft; total time=   0.4s
[CV] END rf__max_depth=7, rf__n_estimators=10, svm__C=9, svm__gamma=0.1, voting=soft; total time=   0.3s
[CV] END rf__max_depth=7, rf__n_estimators=10, svm__C=9, svm__gamma=0.1, voting=hard; total time=   0.2s
[CV] END rf__max_depth=7, rf__n_estimators=10, svm__C=9, svm__gamma=0.1, voting=hard; total time=   0.2s
[CV] END rf__max_depth=7, rf__n_estimators=10, svm__C=9

[CV] END rf__max_depth=7, rf__n_estimators=20, svm__C=9, svm__gamma=0.1, voting=hard; total time=   0.2s
[CV] END rf__max_depth=7, rf__n_estimators=20, svm__C=9, svm__gamma=0.1, voting=hard; total time=   0.2s
[CV] END rf__max_depth=7, rf__n_estimators=20, svm__C=9, svm__gamma=0.1, voting=hard; total time=   0.2s
[CV] END rf__max_depth=7, rf__n_estimators=20, svm__C=9, svm__gamma=0.3, voting=soft; total time=   0.1s
[CV] END rf__max_depth=7, rf__n_estimators=20, svm__C=9, svm__gamma=0.3, voting=soft; total time=   0.1s
[CV] END rf__max_depth=7, rf__n_estimators=20, svm__C=9, svm__gamma=0.3, voting=soft; total time=   0.1s
[CV] END rf__max_depth=7, rf__n_estimators=20, svm__C=9, svm__gamma=0.3, voting=hard; total time=   0.1s
[CV] END rf__max_depth=7, rf__n_estimators=20, svm__C=9, svm__gamma=0.3, voting=hard; total time=   0.1s
[CV] END rf__max_depth=7, rf__n_estimators=20, svm__C=9, svm__gamma=0.3, voting=hard; total time=   0.1s
[CV] END rf__max_depth=7, rf__n_estimators=20, svm__C=9

GridSearchCV(cv=3,
             estimator=VotingClassifier(estimators=[('logistic',
                                                     LogisticRegression()),
                                                    ('svm',
                                                     SVC(probability=True)),
                                                    ('rf',
                                                     RandomForestClassifier())],
                                        voting='soft', weights=[2, 3, 1]),
             n_jobs=1,
             param_grid={'rf__max_depth': [3, 5, 7],
                         'rf__n_estimators': [5, 10, 20],
                         'svm__C': [3, 5, 7, 9], 'svm__gamma': [0.1, 0.3, 0.5],
                         'voting': ['soft', 'hard']},
             scoring='accuracy', verbose=2)

In [35]:
model_best_estimator=model_grid.best_estimator_

In [36]:
model_grid.best_score_

0.7805386031192483

## Saving the model


In [37]:
pickle.dump(model_best_estimator,open('./models/machinelearning_face_person_identity.pkl',mode='wb'))